In [3]:
import os, math, numpy as np
import os
from transformers import AutoTokenizer
import pandas as pd
from tqdm import tqdm
import re, gc
import torch
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
pd.set_option('display.max_rows', 300)
from hydra import compose, initialize

from pathlib import Path

with initialize(config_path="config", version_base=None):
    cfg = compose(config_name="config")
    cfg.exp_number = Path().resolve().name


In [4]:

import numpy as np
def apk(actual, predicted, k=25):
    """
    Computes the average precision at k.
    
    This function computes the average prescision at k between two lists of
    items.
    
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
        
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    
    if not actual:
        return 0.0

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k=25):
    """
    Computes the mean average precision at k.
    
    This function computes the mean average prescision at k between two lists
    of lists of items.
    
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
        
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])


In [6]:
%pip install -qq polars==1.7.1
%pip install -qq datasets==3.0.0
%pip install -qq sentence_transformers==3.1.0


/root/my_kaggle_env/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
/root/my_kaggle_env/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
/root/my_kaggle_env/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
model_path = "/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1"
tokenizer = AutoTokenizer.from_pretrained(model_path)

df_train = pd.read_csv(cfg.data.train_path)
df_test = pd.read_csv(cfg.data.test_path)


OSError: Incorrect path_or_model_id: '/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1'. Please provide either the path to a local folder or the repo_id of a model on the Hub.